In [1]:
import pandas
import ray.dataframe as pd
import ray
import numpy as np

import sys

Process STDOUT and STDERR is being redirected to /tmp/raylogs/.
Waiting for redis server at 127.0.0.1:42368 to respond...
Waiting for redis server at 127.0.0.1:61285 to respond...
Starting local scheduler with the following resources: {'GPU': 0, 'CPU': 4}.

View the web UI at http://localhost:8889/notebooks/ray_ui1901.ipynb?token=af658b219a765e84cd3f205630c483777bbf355cbe981b67



In [2]:
frame_data = np.random.randint(0,100,size=(10**6, 100))
print("Size of data in MB: {}".format(sys.getsizeof(frame_data)/1024/1024))

Size of data in MB: 762.9395599365234


In [3]:
ray_df = pd.DataFrame(frame_data)

Time taken to construct pandas Dataframe: 0.0009381720001329086
Time taken to create blocks via pandas DataFrame: 0.8288855499999954
Time taken to create row_metadata: 0.011016202000064368
Time taken to create col_metadata: 0.002499400000033347
Total constructor time elapsed: 0.8444140920000791


In [4]:
class SuppressPrints:
    def __enter__(self):
        self._original_stdout = sys.stdout
        sys.stdout = None

    def __exit__(self, exc_type, exc_val, exc_tb):
        sys.stdout = self._original_stdout

In [5]:
%%timeit
with SuppressPrints():
    ray_df = pd.DataFrame(frame_data)
    dtypes = ray_df.dtypes

The slowest run took 5.25 times longer than the fastest. This could mean that an intermediate result is being cached.
3.68 s ± 1.99 s per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%timeit
with SuppressPrints():
    ray_df = pd.DataFrame(frame_data)
    dtypes = ray_df._dtypes_cache

The slowest run took 6.96 times longer than the fastest. This could mean that an intermediate result is being cached.
1.17 s ± 977 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%%timeit
pandas.DataFrame(frame_data).dtypes

258 µs ± 15.8 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)
